In [ ]:
from pathlib import Path

import numpy as np
import torch
from torch import Tensor
import pandas as pd
import turbo_broccoli as tb
import seaborn as sns
import bokeh.plotting as bk
import bokeh.layouts as bkl
from bokeh.io import output_notebook, export_png
import matplotlib.pyplot as plt

from nlnas import *

output_notebook()

# Evaluation UMAPS

In [ ]:
model_name = "resnet18"
dataset_name = "cifar10"
version = 0
epoch = 28

path = Path("out") / model_name / dataset_name / f"version_{version}" / str(epoch)

data_plts = tb.load(path / "umap" / "plots.json")
figures = list(data_plts.values())
for fig in figures:
    fig.title.text = ""
    fig.toolbar_location = None
    fig.xgrid.visible = False
    fig.ygrid.visible = False
    fig.axis.visible = False
    fig.height = 250
    fig.width = 250
figure = bkl.row(figures)
export_png(figure, filename="out/resnet18_cifar10.png")
bk.show(figure)

# More than one cluster per class

In [ ]:
model_name = "alexnet"
dataset_name = "fashionmnist"
version = 0
epoch = 25
layer = "model.classifier.4"

path = Path("out") / model_name / dataset_name / f"version_{version}" / str(epoch)
data = tb.load(path / "umap" / "plots.json")
fig_all = data[layer]

In [ ]:
data_eval = tb.load(path / "eval" / "eval.json")
z, y_true = data_eval["z"][layer].flatten(1).numpy(), data_eval["y_true"].numpy()

data_umap = tb.load(path / "umap" / "umap.st")
e = data_umap[layer]

In [ ]:
from bokeh import palettes as bkp
from nlnas.plotting import class_scatter

size, cls = 250, 9
color = bkp.viridis(10)[cls]

fig_1 = bk.figure()
class_scatter(fig_1, e, y_true == cls, palette=["#ffffff", color])

In [ ]:
from bokeh import layouts as bkl

for f in [fig_all, fig_1]:
    f.title.text = ""
    f.toolbar_location = None
    f.xgrid.visible = False
    f.ygrid.visible = False
    f.axis.visible = False
    f.height = 250
    f.width = 250
figure = bkl.row([fig_all, fig_1])
export_png(figure, filename="out/alexnet_fashionmnist_umap.png")
bk.show(figure)

## Trying to manually find samples from each clusters

In [ ]:
idx_a = ((e[:,1] > .7) & (y_true == cls)).nonzero()[0][0]
idx_b = ((e[:,1] < .2) & (y_true == cls)).nonzero()[0][9]

a = data_eval["x"][idx_a]
b = data_eval["x"][idx_b]

In [ ]:
from tqdm.notebook import tqdm
from torchvision.datasets import FashionMNIST
from torchvision import transforms

from nlnas.tv_dataset import TorchvisionDataset
from nlnas.transforms import EnsureRGB, dataset_normalization

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        EnsureRGB(),
        dataset_normalization("fashionmnist"),
        transforms.Resize([64, 64], antialias=True),
    ]
)
ds = FashionMNIST(
    root="/home/cedric/torchvision/datasets/",
    transform=transform,
    train=True,
)

i_a, i_b = -1, -1
for i, (x, y) in enumerate(tqdm(ds)):
    if i_a >= 0 and i_b >= 0:
        break
    if i_a < 0 and torch.isclose(x, a, atol=1).all():
        i_a = i
        print("i_a =", i_a)
    if i_b < 0 and torch.isclose(x, b, atol=1).all():
        i_b = i
        print("i_b =", i_b)

In [ ]:
transform = transforms.Compose(
    [
        # transforms.ToTensor(),
        # EnsureRGB(),
        transforms.Resize([128, 128], antialias=True),
    ]
)
ds = FashionMNIST(
    root="/home/cedric/torchvision/datasets/",
    transform=transform,
    train=True,
)

img_a, img_b = ds[i_a][0], ds[i_b][0]

In [ ]:
img_a

## Multiple clusters per class: super pathological edition

In [ ]:
model_name = "alexnet"
dataset_name = "pcam"
k, bs, we = 5, 2048, 3
version = 0
epoch = 17

experiment = f"{model_name}_finetune_l{k}_b{bs}_1e-{we}"
path = Path("out") / experiment / dataset_name / f"version_{version}" / str(epoch)

data_plts = tb.load(path / "umap" / "plots.json")
figures = list(data_plts.values())
for fig in figures:
    fig.title.text = ""
    fig.toolbar_location = None
    fig.xgrid.visible = False
    fig.ygrid.visible = False
    fig.axis.visible = False
    fig.height = 250
    fig.width = 250
    
figure = bkl.grid([figures[:-3], figures[-3:] + ([None] * 2)])
export_png(figure, filename=f"out/{experiment}_umap.png")
bk.show(figure)

# UMAP of a latent space throughout training

In [ ]:
from nlnas.training import best_epoch

model_name = "googlenet"
dataset_name = "cifar10"
version = 0

be = best_epoch(
    Path("out") / model_name / dataset_name / "model" / "csv_logs" / model_name / f"version_{version}" / "metrics.csv"
)
print("Best epoch:", be)

In [ ]:
n_epoch = 5
layer = "model.inception5b"

epochs = np.linspace(0, be, num=n_epoch, dtype=int)
print("Epochs:", epochs)

path = Path("out") / model_name / dataset_name / f"version_{version}"
all_e = [
    tb.load(path / str(e) / "umap" / "umap.st")[layer] for e in epochs
]
all_eval_data = [
    tb.load(path / str(e) / "eval" / "eval.json") for e in epochs
]
all_z = [d["z"][layer] for d in all_eval_data]
all_y_true = [d["y_true"] for d in all_eval_data]

In [ ]:
from nlnas.clustering import louvain_loss

for z, y_true in zip(all_z, all_y_true):
    print(louvain_loss(z, y_true))

In [ ]:
figures = []
for e, y_true in zip(all_e, all_y_true):
    figure = bk.figure(height = 250, width = 250)
    figure.title.text = ""
    figure.toolbar_location = None
    figure.xgrid.visible = False
    figure.ygrid.visible = False
    figure.axis.visible = False
    class_scatter(figure, z, y_true)
    figures.append(figure)

figure = bkl.row(figures)
bk.show(figure)

# Louvain loss stagnates

In [ ]:
model_name = "alexnet"
dataset_name = "cifar10"
k, bs, we = 5, 2048, 3
version = 0

experiment = f"{model_name}_finetune_l{k}_b{bs}_1e-{we}"
be = best_epoch(
    Path("out") / experiment / dataset_name / "model" / "csv_logs" / experiment / f"version_{version}" / "metrics.csv"
)
print("Best epoch:", be)

In [ ]:
path = Path("out") / experiment / dataset_name / f"version_{version}"

layer = "model.classifier.4"

all_eval_data = [
    tb.load(path / str(e) / "eval" / "eval.json") 
    for e in range(be)
]
all_z = [d["z"][layer] for d in all_eval_data]
all_y_true = [d["y_true"].numpy() for d in all_eval_data]

In [ ]:
from nlnas.clustering import *

df = pd.DataFrame(columns=["epoch", "loss", "miss"])
for i, (z, y_true) in enumerate(zip(all_z, all_y_true)):
    _, y_louvain = louvain_communities(z)
    matching = class_otm_matching(y_true, y_louvain)
    loss = float(clustering_loss(z, y_true, y_louvain, matching, k))
    p1, p2, p3, p4 = otm_matching_predicates(y_true, y_louvain, matching)
    miss = int(p3.sum()) / len(z)
    print("Epoch", i, "/", be, "loss", loss, "miss", miss)
    df.loc[len(df)] = {"epoch": i, "loss": loss, "miss": miss}

df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 

fig, ax1 = plt.subplots() 
ax2 = ax1.twinx() 

sns.lineplot(df, x="epoch", y="loss", ax=ax1, color="red")
sns.lineplot(df, x="epoch", y="miss", ax=ax2, color="blue")

ax1.set_xlabel("Epoch")
ax1.set_ylabel("$\\mathcal{L}_{\\mathrm{Louvain}}$", color="red")
ax2.set_ylabel("$N_{\\mathrm{miss}} / N$", color="blue")

plt.show()
fig.savefig(f"out/{experiment}_{dataset_name}_loss_vs_miss.png")

# LaTeX tables

In [ ]:
from glob import glob
import regex as re
from loguru import logger as logging

all_dfs = []

for path in glob("out/*"):
    experiment_name = path.split("/")[-1]
    if m := re.match(r"^([^_\.]+)$", experiment_name):
        model, k, batch_size, weight = m.group(1), 0, 0, 0
        # print(path, "-> standard", model)
    elif m := re.match(r"^(\w+?)_finetune_l(\d+)_b(\d+)_1e-(\d+)$", experiment_name):
        model, k, batch_size, weight = m.group(1), int(m.group(2)), int(m.group(3)), int(m.group(4))
        # print(path, "-> finetune", model, k, batch_size, weight)
    else:
        print("Skipping", path)
    # weight_str = (
    #     "$10^{-" + str(weight) + "}$"
    #     if int(weight) > 0
    #     else "$1$"
    # )
    for subpath in glob(path + "/*"):
        dataset_name = subpath.split("/")[-1]
        metrics_path = subpath + "/model/csv_logs/" + experiment_name + "/version_0/metrics.csv"
        try:
            df = pd.read_csv(metrics_path)
            df = df[~df["val/acc"].isna()]
            df = df[["val/acc"]]
            df["model"] = model
            df["k"] = k
            df["batch_size"] = batch_size
            df["weight"] = 10 ** (-weight)
            # df["weight_str"] = weight_str
            df["dataset"] = dataset_name
            all_dfs.append(df.loc[[df["val/acc"].idxmax()]])
        except Exception as e:
            print("ERROR:", metrics_path, type(e), str(e))
            # logging.error("Could not load {}: {} {}", metrics_path, e.__class__.__name__, str(e))

In [ ]:
df = pd.concat(all_dfs, ignore_index=True)
df = df.astype({"model": "category", "dataset": "category"})
df = df.sort_values(["model", "dataset", "weight", "k"])
# df["val/acc_str"] = "$" + (df["val/acc"] * 100).round(2).astype(str) + "\%$"

df_orig = df[df["k"] == 0]
for i in range(len(df_orig)):
    r = df_orig.iloc[i]
    idx = (df["model"] == r["model"]) & (df["dataset"] == r["dataset"])
    # gain = df.loc[idx, "val/acc"] / r["val/acc"] - 1
    gain = df.loc[idx, "val/acc"] - r["val/acc"]
    df.loc[idx, "gain"] = gain
    # df.loc[idx, "gain_str"] = "$" + gain.round(2).astype(str) + "\%$"

# df = df[["model", "dataset", "weight", "weight_str", "k", "val/acc", "val/acc_str", "gain", "gain_str"]]
df = df[["model", "dataset", "weight", "k", "val/acc", "gain"]]
df = df[
    df["dataset"].isin(
        [
            "cifar10", 
            "cifar100", 
            "mnist", 
            "fashionmnist", 
            "stl10", 
            "pcam", 
            "eurosat", 
            "semeion"
        ]
    )
]
df

In [ ]:
def float_to_percent(series: pd.Series) -> pd.Series:
    """Converts a series of float e.g. `0.423` to a percentage LaTeX string `$42.3\%$`"""
    return "$" + (series * 100).round(2).astype(str) + "\%$"

def weight_to_tex(series: pd.Series) -> pd.Series:
    """Converts a series of powers of 10 to LaTeX"""
    def _f(x: float) -> str:
        if x == 1:
            return "$1$"
        e = int((np.log(x) / np.log(10)).round(1))
        return "$10^{" + str(e) + "}$"
    return series.apply(_f)

real_column_names = {
    "model": "Model", 
    "dataset": "Dataset", 
    "weight": "$w$", 
    "k": "$k$", 
    "val/acc": "Acc.", 
    "gain": "Gain",
    "tti": "FT/PT",
}
real_model_names = {
    "alexnet": "AlexNet",
    "resnet18": "ResNet-18",
    "googlenet": "GoogLeNet",
    "vgg16": "VGG-16", 
}
real_ds_names = {
    "cifar10": "CIFAR-10",
    "cifar100": "CIFAR-100",
    "mnist": "MNIST",
    "fashionmnist": "Fashion-MNIST",
    "stl10": "STL-10",
    "pcam": "PCam",
    "imagenet": "ImageNet",
    "eurosat": "EuroSAT",
    "semeion": "SEMEION"
}
short_ds_names = {
    "cifar10": "CIF.10",
    "cifar100": "CIF.100",
    "mnist": "MNIST",
    "fashionmnist": "Fashion",
    "stl10": "STL10",
    "pcam": "PCam",
    "imagenet": "ImgNet",
    "eurosat": "Eu.SAT",
    "semeion": "SEM."
}

In [ ]:
df_orig = df[df["k"] == 0][["model", "dataset", "val/acc"]]
df_orig.set_index(["model", "dataset"], inplace=True)

df_orig_camera = df_orig.copy()
df_orig_camera["val/acc"] = float_to_percent(df_orig_camera["val/acc"])
df_orig_camera.rename(columns=real_column_names, inplace=True)
df_orig_camera.index.names = [real_column_names[s] for s in df_orig_camera.index.names]
df_orig_camera.rename(index=real_model_names, level=0, inplace=True)
df_orig_camera.rename(index=real_ds_names, level=1, inplace=True)
df_orig_camera

In [ ]:

idx = df[(df["k"] != 0)].groupby(["model", "dataset"])["val/acc"].idxmax().dropna()
df_finetune_best = df.loc[idx][["model", "dataset", "weight", "k", "val/acc", "gain"]]
df_finetune_best.set_index(["model", "dataset"], inplace=True)

df_finetune_best_camera = df_finetune_best.copy()
# df_finetune_best_camera = df_finetune_best_camera.style.background_gradient(subset="gain")
df_finetune_best_camera["val/acc"] = float_to_percent(df_finetune_best_camera["val/acc"])
df_finetune_best_camera["gain"] = float_to_percent(df_finetune_best_camera["gain"])
df_finetune_best_camera["weight"] = weight_to_tex(df_finetune_best_camera["weight"])
df_finetune_best_camera["k"] = df_finetune_best_camera["k"].apply(lambda s: f"${s}$")
df_finetune_best_camera.rename(columns=real_column_names, inplace=True)
df_finetune_best_camera.index.names = [real_column_names[s] for s in df_finetune_best_camera.index.names]
df_finetune_best_camera.rename(index=real_model_names, level=0, inplace=True)
df_finetune_best_camera.rename(index=real_ds_names, level=1, inplace=True)
# df_finetune_best_camera.rename(index=short_ds_names, level=1, inplace=True)
df_finetune_best_camera

In [ ]:
from itertools import product

df_hparm = pd.DataFrame(columns=["k", "weight", "avg_gain"])

for k, w in product(np.unique(df["k"]), np.unique(df["weight"])):
    if k == 0 or w == 0:
        continue
    df_hparm.loc[len(df_hparm)] = {
        "k": k,
        "weight": w,
        "avg_gain": df[(df["k"] == k) & (df["weight"] == w)]["gain"].mean(skipna=True)
    }
df_hparm.sort_values("avg_gain", inplace=True, ascending=False)
df_hparm

In [ ]:
idx = (
    df[
        (df["gain"] > 0) 
        & (df["k"] == 5) 
        & (df["weight"] == 1e-3)
    ]
    .groupby(["model", "dataset"])["val/acc"]
    .idxmax()
    .dropna()
)
df_finetune_default = df.loc[idx][["model", "dataset", "val/acc", "gain"]]
df_finetune_default.set_index(["model", "dataset"], inplace=True)
df_finetune_default["vs_best"] = df_finetune_default["gain"] - df_finetune_best["gain"]

df_finetune_default_camera = df_finetune_default.copy()
df_finetune_default_camera["val/acc"] = float_to_percent(df_finetune_default_camera["val/acc"])
df_finetune_default_camera["gain"] = float_to_percent(df_finetune_default_camera["gain"])
df_finetune_default_camera["vs_best"] = float_to_percent(df_finetune_default_camera["vs_best"])
df_finetune_default_camera.rename(columns=real_column_names, inplace=True)
df_finetune_default_camera.rename(columns={"vs_best": "Vs. best"}, inplace=True)
df_finetune_default_camera.index.names = [real_column_names[s] for s in df_finetune_default_camera.index.names]
df_finetune_default_camera.rename(index=real_model_names, level=0, inplace=True)
df_finetune_default_camera.rename(index=real_ds_names, level=1, inplace=True)
# df_finetune_default_camera.rename(index=short_ds_names, level=1, inplace=True)

df_finetune_default_camera

In [ ]:
df_wall = pd.read_csv("out/wall.csv")
df_wall.drop("Unnamed: 0", axis=1, inplace=True)
df_wall = df_wall.groupby(["model", "dataset", "k", "weight"]).mean()
df_wall.drop(["step", "epoch"], axis=1, inplace=True)

df_finetune_tti = df.copy()
df_finetune_tti.reset_index(inplace=True)
df_finetune_tti.set_index(["model", "dataset", "k", "weight"], inplace=True)
# df_finetune_tti.drop(["val/acc", "gain"], axis=1, inplace=True)
df_finetune_tti = df_finetune_tti.join(df_wall, how="inner")
df_finetune_tti.reset_index(inplace=True)
df_finetune_tti = df_finetune_tti[df_finetune_tti["k"] > 0]
# df_finetune_tti.set_index(["model", "dataset"], inplace=True)

df_wall.reset_index(inplace=True)
df_wall_orig = df_wall[df_wall["k"] == 0]
for i in range(len(df_wall_orig)):
    r = df_wall_orig.iloc[i]
    idx = (
        (df_finetune_tti["model"] == r["model"]) 
        & (df_finetune_tti["dataset"] == r["dataset"])
    )
    tti = df_finetune_tti.loc[idx, "wall_time"] / r["wall_time"] - 1
    df_finetune_tti.loc[idx, "tti"] = tti
df_finetune_tti = df_finetune_tti[["model", "dataset", "tti"]]
df_finetune_tti = df_finetune_tti.groupby(["model", "dataset"]).mean()

df_finetune_tti_camera = df_finetune_tti.copy()
df_finetune_tti_camera["tti"] = float_to_percent(df_finetune_tti_camera["tti"])
df_finetune_tti_camera.rename(columns=real_column_names, inplace=True)
df_finetune_tti_camera.index.names = [real_column_names[s] for s in df_finetune_tti_camera.index.names]
df_finetune_tti_camera.rename(index=real_model_names, level=0, inplace=True)
df_finetune_tti_camera.rename(index=real_ds_names, level=1, inplace=True)
# df_finetune_tti_camera.rename(index=short_ds_names, level=1, inplace=True)
df_finetune_tti_camera


In [ ]:
kwargs = {
    "multirow_align": "t", 
    "hrules": True,  
    "position_float": "centering"
}
table_size = "tiny"  # or "small"
with open("out/val_acc_orig.tex", "w", encoding="utf-8") as fp:
    latex = df_orig_camera.style.to_latex(
        label="tab:val-acc:original",
        caption="Pretrained model accuracy.",
        position="H",
        **kwargs
    )
    latex = latex.replace(
        "\\begin{tabular}", 
        f"\\vskip 0.15in\n\\{table_size}\n"
        "\\begin{tabular}"
    )
    latex = latex.replace(
        "\\end{tabular}", 
        "\\end{tabular}\n\\vskip -0.1in"
    )
    fp.write(latex)
with open("out/val_acc_finetune_best.tex", "w", encoding="utf-8") as fp:
    latex = df_finetune_best_camera.style.to_latex(
        label="tab:val-acc:finetune-best",
        caption="Best fine-tuning parameters and results.",
        position="t",
        **kwargs
    )
    latex = latex.replace(
        "\\begin{tabular}", 
        (
            f"\\vskip 0.15in\n\\{table_size}\n"
            "\\begin{tabular}"
        )
    )
    latex = latex.replace(
        "\\end{tabular}", 
        "\\end{tabular}\n\\vskip -0.1in"
    )
    fp.write(latex)
with open("out/val_acc_finetune_default.tex", "w", encoding="utf-8") as fp:
    latex = df_finetune_default_camera.style.to_latex(
        label="tab:val-acc:finetune-default",
        caption="Best fine-tuning parameters and results for $k = 5$ and $w = 10^{-3}$.",
        position="H",
        **kwargs
    )
    latex = latex.replace(
        "\\begin{tabular}", 
        (
            f"\\vskip 0.15in\n\\{table_size}\n"
            "\\begin{tabular}"
        )
    )
    latex = latex.replace(
        "\\end{tabular}", 
        "\\end{tabular}\n\\vskip -0.1in"
    )
    fp.write(latex)
with open("out/wall_finetune.tex", "w", encoding="utf-8") as fp:
    latex = df_finetune_tti_camera.style.to_latex(
        label="tab:wall:finetune-all",
        caption=(
            "Average ratio between fine-tuning time (FT) and pretraining time (PT). "
            "For example, a FT/PT value of $100\\%$ means that fine-tuning took "
            "as long as pretraining, on average, across all combinations of "
            "parameters that were tested (see \\cref{ssec:setup})."
        ),
        position="t",
        **kwargs
    )
    latex = latex.replace(
        "\\begin{tabular}", 
        (
            f"\\vskip 0.15in\n\\{table_size}\n"
            "\\begin{tabular}"
        )
    )
    latex = latex.replace(
        "\\end{tabular}", 
        "\\end{tabular}\n\\vskip -0.1in"
    )
    fp.write(latex)

# Miss-match CE plots

In [ ]:
model_name = "alexnet"
dataset_name = "cifar10"
k, bs, we = 5, 2048, 3
version = 0

# experiment = model_name
experiment = f"{model_name}_finetune_l{k}_b{bs}_1e-{we}"

path = Path("out") / experiment / dataset_name / f"version_{version}" / "metrics.csv"
df = pd.read_csv(path)
df = df[["epoch", "layer", "acc_all", "acc_miss", "acc_match"]]
df.rename(columns={"acc_all": "all", "acc_miss": "miss", "acc_match": "match"}, inplace=True)
df = df.melt(
    id_vars=["epoch", "layer"],
    var_name="subset",
    value_name="acc",
)

real_subset_names = {"all": "All", "match": "Correctly clust.", "miss": "Misscl."}
df["subset"] = df["subset"].apply(lambda s: real_subset_names[s])
df["layer"] = df["layer"].apply(lambda s: s[len("model."):])

real_column_names = {
    "epoch": "Epoch",
    "layer": "Layer",
    "subset": "Sample group",
    "acc": "Accuracy",
}
df.rename(columns=real_column_names, inplace=True)

df

In [ ]:
figure = sns.relplot(
    data=df,
    x=real_column_names["epoch"],
    y=real_column_names["acc"],
    hue=real_column_names["subset"],
    palette={
        real_subset_names["all"]: "black", 
        real_subset_names["match"]: "blue", 
        real_subset_names["miss"]: "red"
    },
    style=real_column_names["subset"],
    dashes={
        real_subset_names["all"]: (1, 1),
        real_subset_names["match"]: "", 
        real_subset_names["miss"]: ""
    },
    col=real_column_names["layer"],
    kind="line",
    col_wrap=3,
)

In [ ]:
figure.fig.savefig(f"out/{experiment}_{dataset_name}_cluster_acc.png")
plt.clf()